In [1]:
import torch
import torch.nn as nn
import torch.utils.data as Data
import torchvision
import matplotlib.pyplot as plt
import os
import cv2

D:\anaconda3\envs\pytracking\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
torch.manual_seed(1)  # 使用随机化种子使神经网络的初始化每次都相同

# 超参数
EPOCH = 1  # 训练整批数据的次数
BATCH_SIZE = 32 # 10 20 30, 8 16 32 64 
LR = 0.001  # 学习率

In [3]:
# 下载mnist手写数据集
train_data = torchvision.datasets.MNIST(
    root='./data/',  # 保存或提取的位置  会放在当前文件夹中
    train=True,  # true说明是用于训练的数据，false说明是用于测试的数据
    transform=torchvision.transforms.ToTensor(),  # 转换PIL.Image or numpy.ndarray
    download=True,
)

9920512it [00:25, 386733.22it/s]                                                                                       


Extracting ./data/MNIST\raw\train-images-idx3-ubyte.gz to ./data/MNIST\raw


32768it [00:21, 1515.32it/s]                                                                                           


Extracting ./data/MNIST\raw\train-labels-idx1-ubyte.gz to ./data/MNIST\raw


1654784it [00:23, 71530.41it/s]                                                                                        


Extracting ./data/MNIST\raw\t10k-images-idx3-ubyte.gz to ./data/MNIST\raw


8192it [00:21, 377.53it/s]                                                                                             
D:\anaconda3\envs\pytracking\lib\site-packages\torchvision\datasets\mnist.py:469: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


Extracting ./data/MNIST\raw\t10k-labels-idx1-ubyte.gz to ./data/MNIST\raw
Processing...
Done!


In [4]:
print(train_data)

Dataset MNIST
    Number of datapoints: 60000
    Root location: ./data/
    Split: Train
    StandardTransform
Transform: ToTensor()


In [6]:
test_data = torchvision.datasets.MNIST(
    root='./data/',
    train=False  # 表明是测试集
)

In [7]:
print(test_data)

Dataset MNIST
    Number of datapoints: 10000
    Root location: ./data/
    Split: Test


In [8]:
# 进行测试
# 为节约时间，测试时只测试前2000个
test_x = torch.unsqueeze(test_data.train_data, dim=1).type(torch.FloatTensor)[:2000] / 255 
# torch.unsqueeze(a) 是用来对数据维度进行扩充，这样shape就从(2000,28,28)->(2000,1,28,28) 
# 图像的pixel本来是0到255之间，除以255对图像进行归一化使取值范围在(0,1)
test_y = test_data.test_labels[:2000]

D:\anaconda3\envs\pytracking\lib\site-packages\torchvision\datasets\mnist.py:55: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")
D:\anaconda3\envs\pytracking\lib\site-packages\torchvision\datasets\mnist.py:50: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


In [13]:
# 批训练 32个samples， 1  channel，28x28 (50,1,28,28)
# Torch中的DataLoader是用来包装数据的工具，它能帮我们有效迭代数据，这样就可以进行批训练
train_loader = Data.DataLoader(
    dataset=train_data,
    batch_size=BATCH_SIZE,
    shuffle=True  # 是否打乱数据，一般都打乱
)

In [9]:

class CNN(nn.Module):  # 我们建立的CNN继承nn.Module这个模块
    def __init__(self):
        super(CNN, self).__init__()
        # 建立第一个卷积(Conv2d)-> 激励函数(ReLU)->池化(MaxPooling)
        self.conv1 = nn.Sequential(
            # 第一个卷积con2d
            nn.Conv2d(  # 输入图像大小(1,28,28)
                in_channels=1,  # 输入图片的高度，因为minist数据集是灰度图像只有一个通道
                out_channels=16,  # n_filters 卷积核的高度
                kernel_size=5,  # filter size 卷积核的大小 也就是长x宽=5x5  3*3 3*3 3*3 1*3+3*1
                stride=1,  # 步长
                padding=2,  # 想要con2d输出的图片长宽不变，就进行补零操作 padding = (kernel_size-1)/2
            ),  # 输出图像大小(16,28,28)
            # 激活函数
            nn.ReLU(), # Sigmoid
            # 池化，下采样
            nn.MaxPool2d(kernel_size=2),  # 在2x2空间下采样
            # 输出图像大小(16,14,14)
        )
        # 建立第二个卷积(Conv2d)-> 激励函数(ReLU)->池化(MaxPooling)
        self.conv2 = nn.Sequential(
            # 输入图像大小(16,14,14)
            nn.Conv2d(  # 也可以直接简化写成nn.Conv2d(16,32,5,1,2)
                in_channels=16,
                out_channels=32,
                kernel_size=5,
                stride=1,
                padding=2
            ),
            # 输出图像大小 (32,14,14)
            nn.ReLU(),
            nn.MaxPool2d(2),
            # 输出图像大小(32,7,7)
        )
        # 建立全卷积连接层
        self.out = nn.Linear(32 * 7 * 7, 10)  # 输出是10个类

         # Init weights
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
                if m.bias is not None:
                    m.bias.data.zero_()
    # 下面定义x的传播路线
    def forward(self, x):
#         x = self.conv1(x)  # x先通过conv1
#         x = self.conv2(x)  # 再通过conv2
        x = self.conv2(self.conv1(x))# 32 7 7 1 32*7*7
        # 把每一个批次的每一个输入都拉成一个维度，即(batch_size,32*7*7)
        # 因为pytorch里特征的形式是[bs,channel,h,w]，所以x.size(0)就是batchsize
        x = x.view(x.size(0), -1)  # view就是把x弄成batchsize行个tensor reshape view
        output = self.out(x) #32 32*7*7
        return output #32 * 10

In [10]:
cnn = CNN()
print(cnn)

CNN(
  (conv1): Sequential(
    (0): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (out): Linear(in_features=1568, out_features=10, bias=True)
)


In [16]:
def cnn_paras_count(net):
    """cnn参数量统计"""
    # Find total parameters and trainable parameters
    total_params = sum(p.numel() for p in net.parameters())
    print("Number of total parameter: %.2fM" % (total_params / 1e6))
    total_trainable_params = sum(p.numel() for p in net.parameters() if p.requires_grad)
    print("Number of trainable parameter: %.2fM" % (total_trainable_params / 1e6))

In [ ]:
cnn_paras_count(cnn)

In [26]:
# 训练
# 把x和y 都放入Variable中，然后放入cnn中计算output，最后再计算误差

In [11]:
# 优化器选择Adam
optimizer = torch.optim.Adam(cnn.parameters(), lr=LR) #SGD adam adamW 
# 损失函数 
loss_func = nn.CrossEntropyLoss()  # 目标标签是one-hotted

In [15]:
#traing
for epoch in range(EPOCH):
    for step, (b_x, b_y) in enumerate(train_loader):  # 分配batch data
        output = cnn(b_x)  # 先将数据放到cnn中计算output
        loss = loss_func(output, b_y)  # 输出和真实标签的loss，二者位置不可颠倒
        optimizer.zero_grad()  # 清除之前学到的梯度的参数
        loss.backward()  # 反向传播，计算梯度
        optimizer.step()  # 应用梯度

        if step % 32 == 0:
            test_output = cnn(test_x)
            pred_y = torch.max(test_output, 1)[1].data.numpy()
            accuracy = float((pred_y == test_y.data.numpy()).astype(int).sum()) / float(test_y.size(0))
            print('Epoch: ', epoch+1, '| train loss: %.4f' % loss.data.numpy(), '| test accuracy: %.2f' % accuracy)

torch.save(cnn.state_dict(), 'cnn.pkl')#保存模型

Epoch:  1 | train loss: 0.0472 | test accuracy: 0.98
Epoch:  1 | train loss: 0.0193 | test accuracy: 0.98
Epoch:  1 | train loss: 0.0020 | test accuracy: 0.98
Epoch:  1 | train loss: 0.0137 | test accuracy: 0.98
Epoch:  1 | train loss: 0.0138 | test accuracy: 0.98
Epoch:  1 | train loss: 0.2101 | test accuracy: 0.98
Epoch:  1 | train loss: 0.0018 | test accuracy: 0.98
Epoch:  1 | train loss: 0.0199 | test accuracy: 0.98
Epoch:  1 | train loss: 0.0940 | test accuracy: 0.98
Epoch:  1 | train loss: 0.0203 | test accuracy: 0.98
Epoch:  1 | train loss: 0.0210 | test accuracy: 0.98
Epoch:  1 | train loss: 0.0264 | test accuracy: 0.98
Epoch:  1 | train loss: 0.0623 | test accuracy: 0.98
Epoch:  1 | train loss: 0.0578 | test accuracy: 0.98
Epoch:  1 | train loss: 0.0376 | test accuracy: 0.98
Epoch:  1 | train loss: 0.0381 | test accuracy: 0.98
Epoch:  1 | train loss: 0.0327 | test accuracy: 0.98
Epoch:  1 | train loss: 0.0269 | test accuracy: 0.98
Epoch:  1 | train loss: 0.0186 | test accuracy

In [29]:
# 加载模型，调用时需将前面训练及保存模型的代码注释掉，否则会再训练一遍
cnn.load_state_dict(torch.load('cnn.pkl'))
cnn.eval()
# print 10 predictions from test data
inputs = test_x[:32]  # 测试32个数据
test_output = cnn(inputs)
pred_y = torch.max(test_output, 1)[1].data.numpy()
print(pred_y, 'prediction number')  # 打印识别后的数字
# print(test_y[:10].numpy(), 'real number')

img = torchvision.utils.make_grid(inputs)
img = img.numpy().transpose(1, 2, 0)

# 下面三行为改变图片的亮度
# std = [0.5, 0.5, 0.5]
# mean = [0.5, 0.5, 0.5]
# img = img * std + mean
cv2.imshow('win', img)  # opencv显示需要识别的数据图片
key_pressed = cv2.waitKey(0)

[7 2 1 0 4 1 4 9 5 9 0 6 9 0 1 5 9 7 3 4 9 6 6 5 4 0 7 4 0 1 3 1] prediction number


KeyboardInterrupt: 